# Agenda, day 2

1. Recap and Q&A
2. Magic methods / "dunder" methods
3. Class attributes
4. Finding attributes with ICPO
5. Inheritance -- what it is, and how ICPO influences it
6. Three models for method inheritance
7. Data inheritance
8. What next?

# Recap

How do we solve problems? It's easiest if we can use a data structure that is appropriate for solving that problem. Objects allow us to create a data structure that is specifically designed to solve a problem. Even though we might be using ints, strings, lists, and dicts to implement our object, the fact that we can think about it at a higher level makes it easier to work with.

The whole idea of object-oriented programming is thus: Create new data structures, along with methods, that together make it easier to think about and use these structures and thus solve our problems.

1. Methods are defined as part of the class body. And actually, methods are attributes on the class. However, we invoke them via the instance, and Python does a little magical substitution, turning the instance into the first argument passed to the method.
2. The first parameter in every method is traditionally called `self`, and gets that instance assigned to it.
3. If you try to invoke a method that doesn't exist, you'll get an "attribute error," with Python telling you that the method doesn't exist.
4. We define new "classes," factories for our data structures. We start the class definition with the reserved word `class`, then the name we want to give it, and then any number of methods that we might want to invoke on our data.
5. If we have a `Person` class, then it creates *instances* of `Person`. Every object has a `type`, and we can invoke `type()` on it to find out its class. In this way, `class` and 